# ch02 싱글턴 패턴을 사용해 유일 객체 생성하기

- 프로그램이 실행되는 동안 인스턴스(클래스의 인스턴스, 리스트, 딕셔너리 등)를 하나만 만들어야 할 때가 있음
- 대개 둘 이상의 인스턴스가 생기면 로직상 에러가 발생하는 등의 이유로 하나만 원하는 것
- 인스턴스를 하나만 생성하도록 강제하는 디자인 패턴을 singleton 이라고 부름
- 모듈 레벨, 클래식, borg 싱글턴에 대해 배움 
- 또한 이들이 어떻게 동작하는지, 언제 사용하는지 등과 공유 자원에서 하나의 싱글턴을 사용하는 멀티 스레드 웹 크롤러를 만들어 볼 예정

### 싱글턴 적합 상황

- 공유 자원에 대한 동시 접근을 제어할 필요가 있을 때
- 여러 시스템에서 하나의 자원에 접근하는 지점이 필요할 때
- 유일 객체가 필요할 때

### 싱글턴 사용 예

- 로깅 클래스와 서브클래스(로그에 메시지를 보내기 위한 로깅 클래스에 대한 글로벌 접근 지점)
- 프린트 스플러(spooler): 동일 자원에 대한 요청 간섭을 피하기 위해서 하나의 스풀러만 사용해야 함
- 데이터 베이스 연결 관리
- 파일 관리자
- 외부 설정 파일에서 정보를 얻거나 저장할 때
- 전역 상태(사용자 언어, 시간, 시간대, 애플리케이션 경로 등)를 담고 있는 읽기 전용 싱글턴

## 모듈 레벨 싱글턴

1. 모듈이 이미 임포트되었는지 확인한다
2. 그렇다면 그 모듈을 반환한다
3. 임포트되지 않았다면 초기화 한 후 반환한다.
4. 모듈 초기화는, 모든 모듈 레벨 할당을 포함한 코드 실행을 의미함. 모듈을 처음으로 임포트하면 모든 초기화가 수행됨. 하지만 모듈을 두번째로 임포트하려고 하면 파이썬이 이미 초기화된 모듈을 반환함

In [2]:
%%writefile src/singletona.py

only_one_var = "I'm only one var"

Writing src/singletone.py


In [4]:
%%writefile src/module1.py

import singleton
print singleton.only_one_var
singleton.only_one_var += ' after modification'
import module2

Writing src/module1.py


In [5]:
%%writefile src/module2.py

import singleton
print singleton.only_one_var

Writing src/module2.py


In [25]:
!python src/module1.py

I'm only one var
I'm only one var after modification


- module1을 실행시켜 보면 singleton.py 모듈의 전역 변수를 임포트하고 이 값을 module1.py에서 변경하면, module2.py에서도 변동된 변수를 받음

### 문제점

- 에러가 발생할 확률이 높음. 예를 들어, 전역 명령어를 잊는다면 함수에 묶인 지역변수가 생성되고 모듈의 변수가 변경디지 않는 에러가 생김
- 깔끔하지 못하다. 특히 싱글톤으로 유지해야 하는 객체가 많은 경우 지저분해짐
- 모듈 네임스페이스와 함께 불필요한 변수가 생김
- 게으른 할당과 초기화를 할 수 없음. 모든 전역 변수는 모듈을 임포트하는 과정에서 불러옴
- 상속 기능을 사용할 수 없어 코드를 재사용하는 것이 불가능 함
- 특별 메소드와 객체 지향 프로그래밍의 장점을 전혀 살릴 수 없음

## 클래식 싱글톤

- 파이썬의 클래식 싱글톤에서 인스턴스가 이미 생성되었는지 확인함
- 생성되었다면 반환하고, 그렇지 않은 경우 새로운 인스턴스를 만들고 클래스 속성에 할당한 후 반환함

In [27]:
class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance

In [28]:
singleton = Singleton()

In [29]:
another_singleton = Singleton()

In [30]:
singleton is another_singleton

True

In [31]:
singleton.only_one_var = "I'm only one var"

In [32]:
another_singleton.only_one_var

"I'm only one var"

In [33]:
class Child(Singleton):
    pass

In [34]:
child = Child()

In [35]:
# 똑같다고 나오는데..?? 뭐가 문제?
child is singleton

True

In [36]:
child.only_one_var

"I'm only one var"

In [37]:
%%writefile src/singleton_test.py

class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance
    
singleton = Singleton()
another_singleton = Singleton()
print('singleton is another_singleton', singleton is another_singleton)

singleton.only_one_var = "I'm only one var"
print(another_singleton.only_one_var)

class Child(Singleton):
    pass

child = Child()
print('child is singleton', child is singleton)
print('child.only_one_var', child.only_one_var)

Writing src/singleton_test.py


In [38]:
!python src/singleton_test.py

('singleton is another_singleton', True)
I'm only one var
('child is singleton', True)
('child.only_one_var', "I'm only one var")


## 보그 싱글턴

- borg: 단일 상태(monostate)
- 보그 패턴에서는, 모든 인스턴스는 서로 다르지만 동일한 상태를 공유함
- 공유 상태는 \_shared\_state 속성으로 관리함
- Borg의 새로운 인스턴스는 클래스 메소드 \_\_new\_\_에 정의된 대로 이 상태를 갖음

In [80]:
class Borg(object):
    _shared_state = {}
    
    def __new__(cls, *args, **kwargs):
        obj = super(Borg, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._shared_state
        return obj

In [81]:
del Child

In [82]:
class Child(Borg):
    pass

In [83]:
borg = Borg()

In [84]:
another_borg = Borg()

In [85]:
borg is another_borg

False

In [86]:
child = Child()

In [87]:
borg.only_one_var = "I'm the only one var"

In [88]:
child.only_one_var

"I'm the only one var"

- is 명령어를 사용해서 객체가 동일한지 비교할 수 없다는 점을 제외한다면 모든 자녀 객체는 부모 객체의 상태를 공유함


- Borg 클래스의 하위 클래스지만 다른 상태를 갖게 하고 싶다면 다음과 같이 shared\_state를 재설정한다.

In [89]:
class AnotherChild(Borg):
    # 오타 주의하자...
    _shared_state = {}

In [90]:
another_child = AnotherChild()

In [91]:
another_child.only_one_var

AttributeError: 'AnotherChild' object has no attribute 'only_one_var'

### 어떤 싱글톤을 사용할지는 독자의 몫

- 싱글톤을 상속할 일이 없다면 클래식 싱글톤을 사용할 수 있지만
- 그렇지 않다면 보그 싱글톤을 사용하는 것이 알맞다.

## 파이썬 구현

- 실제 예제로, 지정한 웹사이트를 스캔하는 간단한 crawler를 만들어 보자
- 이 크롤러는 동일한 웹사이트로의 링크를 따라가서 발견한 모든 이미지를 다운로드 함
- 첫번째 함수는 방문할 페이지 집합을 만들기 위해 웹사이트를 스캔하는 용도이고, 두번째는 이미지를 찾고 내려받기 위한 용도
- 동작을 빠르게 하기 위해서 이미지 내려받기를 쓰레드 2개로 수행함
- 쓰레드 2개는 상호간섭이 없어야 하므로 1 스레드에서 페이지를 이미 스캔했다면 다른 곳에서는 이를 건너 뛰어서 동일한 이미지가 중복되지 않게 한다.
- 따라서 내려받은 이미지와 스캔한 웹페이지는 애플리케이션에서 공유하는 자원이 되니 싱글톤 인스턴스에서 관리해야 함

In [152]:
# coding: utf-8
import httplib2
import os
import re
import threading
import urllib
from urlparse import urlparse, urljoin
from BeautifulSoup import BeautifulSoup


class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance


class ImageDownloaderThread(threading.Thread):
    """ 병렬적으로 이미지를 다운로드 받는 스레드"""
    def __init__(self, thread_id, name, counter):
        threading.Thread.__init__(self)
        self.name = name

    def run(self):
        print('Starting thread ', self.name)
        download_images(self.name)
        print('Finished thread ', self.name)


def traverse_site(max_links=10):
    link_parser_singleton = Singleton()

    # 큐에 파싱할 페이지가 있는 동안
    while link_parser_singleton.queue_to_parse:
        # 이미지를 내렵다을 링크를 충분히 모은 경우, 반환
        if len(link_parser_singleton.to_visit) == max_links:
            return

        url = link_parser_singleton.queue_to_parse.pop()

        http = httplib2.Http()
        try:
            status, response = http.request(url)
        except Exception:
            continue

        # 웹페이지가 아닌 경우 건너뛴다.
        if not 'text/html' in status.get('content-type'):
            continue

        # 이미지를 내려받기 위해 링크를 큐에 추가한다.
        link_parser_singleton.to_visit.add(url)
        print('Added', url, 'to queue')

        bs = BeautifulSoup(response)

        for link in BeautifulSoup.findAll(bs, 'a'):
            link_url = link.get('href')

            # <img> 태그에 href 속성이 없을 수도 있음
            if not link_url:
                continue

            parsed = urlparse(link_url)

            # 링크가 외부 웹페이지로 연결된다면 건너뜀
            if parsed.netloc and parsed.netloc != parsed_root.netloc:
                continue

            # 상대 경로를 사용한 링크를 전체 경로로 변환함
            link_url = (parsed.scheme or parsed_root.scheme) + '://' + \
            (parsed.netloc or parsed_root.netloc) + parsed.path or ''

            # 중복된 링크일 경우 건너뛴다.
            if link_url in link_parser_singleton.to_visit:
                continue

            # 파싱을 위해 링크를 추가한다.
            link_parser_singleton.queue_to_parse = [link_url] + \
            link_parser_singleton.queue_to_parse


def download_images(thread_name):
    singleton = Singleton()
    while singleton.to_visit:

        url = singleton.to_visit.pop()

        http = httplib2.Http()
        print thread_name, 'Starting downloading images from', url

        try:
            status, response = http.request(url)
        except Exception:
            continue

        bs = BeautifulSoup(response)

        # 모든 <img> 태그를 찾는다.
        images = BeautifulSoup.findAll(bs, 'img')

        for image in images:
            # 절대 혹은 상대 경로 url로 된 이미지 소스를 얻는다.
            src = image.get('src')
            # 전체 url을 만든다. 만약 이미지 url이 상대 경로라면
            # 웹페이지 도메인과 함께 확장된다.
            # url이 절대 경로라면 그대로 진행한다.
            src = urljoin(url, src)

            # 베이스 이름을 얻는다. 예를 들어 로컬 이름 파일에 image.png를 의미한다.
            basename = os.path.basename(src)

            if src not in singleton.downloaded:
                singleton.downloaded.add(src)
                print('Downloading', src)
                # 로컬 파일시스템에서 이미지를 다운로드 받는다.
                urllib.urlretrieve(src, os.path.join('images', basename))

        print(thread_name, 'finished downloading images from', url)


if __name__ == '__main__':
    root = 'http://python.org'
    parsed_root = urlparse(root)
    singleton = Singleton()
    singleton.queue_to_parse = [root]
    # A set of urls to download images from
    singleton.to_visit = set()
    # Downloaded images
    singleton.downloaded = set()

    traverse_site()

    # Create images directory if not exists
    if not os.path.exists('images'):
        os.makedirs('images')

    # Create new threads
    thread1 = ImageDownloaderThread(1, 'Thread-1', 1)
    thread2 = ImageDownloaderThread(2, 'Thread-2', 2)

    # Start new Threads
    thread1.start()
    thread2.start()

('Added', 'http://python.org', 'to queue')
('Added', u'http://python.org/', 'to queue')
('Added', u'http://python.org/psf-landing/', 'to queue')
('Added', u'http://python.org/jobs/', 'to queue')
('Added', u'http://python.org/community/', 'to queue')
('Added', u'http://python.org/', 'to queue')
('Added', u'http://python.org/community/irc/', 'to queue')
('Added', u'http://python.org/accounts/login/', 'to queue')
('Added', u'http://python.org/accounts/signup/', 'to queue')
('Added', u'http://python.org/accounts/login/', 'to queue')
('Added', u'http://python.org/about/', 'to queue')
('Added', u'http://python.org/about/apps/', 'to queue')
('Starting thread ', 'Thread-1')
Thread-1 Starting downloading images from http://python.org/community/irc/
('Starting thread ', 'Thread-2')
Thread-2 Starting downloading images from http://python.org/accounts/signup/
('Downloading', u'http://python.org/static/img/python-logo.png')
('Thread-2', 'finished downloading images from', u'http://python.org/accoun

In [151]:
!python crawler.py

('Starting thread ', 'Thread-1')
 ('Starting thread ', ('Finished thread ', 'Thread-1''Thread-2')
('Finished thread ', 'Thread-2')
)


In [153]:
%ll images/

total 56
-rw-r--r--  1 re4lfl0w  staff  14117  9 26 23:44 psf-logo.png
-rw-r--r--  1 re4lfl0w  staff  10102  9 26 23:44 python-logo.png


In [108]:
urljoin('abc/d/', '1.html')

'abc/d/1.html'

In [107]:
urljoin('naver.com', 'abc/d/', '1.html')

'abc/d/'

In [94]:
urljoin('http', 'naver.com', '1.html')

'naver.com'

In [101]:
urljoin('http://', 'naver.com', '1.html')

'http:///naver.com'

In [112]:
'http://{}{}'.format('naver.com/', urljoin('abc/d/', '1.html'))

'http://naver.com/abc/d/1.html'

In [113]:
url = 'http://naver.com/good/path/1.html'

In [116]:
url_parse = urlparse(url)
url_parse

ParseResult(scheme='http', netloc='naver.com', path='/good/path/1.html', params='', query='', fragment='')

In [115]:
from urlparse import urlunparse

In [131]:
urlunparse((url_parse.scheme, url_parse.netloc, url_parse.path, 
           url_parse.params, url_parse.query, url_parse.fragment))

'http://naver.com/good/path/1.html'

In [133]:
# fragment까지 넣어주지 않으면 조합이 안되네.
# urlparse에서 나온 순서대로 넣어줘야 됨
urlunparse((url_parse.scheme, url_parse.netloc, url_parse.path, 
           url_parse.params, url_parse.query))

ValueError: need more than 5 values to unpack

- [python \- Combining a url with urlunparse \- Stack Overflow](https://stackoverflow.com/questions/3798269/combining-a-url-with-urlunparse/3798311#3798311)

In [129]:
def FixScheme(website):

    from urlparse import urlparse, urlunparse

    scheme, netloc, path, params, query, fragment = urlparse(website)
    print(urlparse(website))
    
    # 팁
    if not netloc:
        netloc, path = path, ''

    if scheme == '':
        return urlunparse(('http', netloc, path, params, query, fragment))
    else:
        return website

In [130]:
FixScheme('www.python.org')

ParseResult(scheme='', netloc='', path='www.python.org', params='', query='', fragment='')


'http://www.python.org'

## 요약

- 싱글톤은 클래스의 인스턴스를 하나만 만들 때 사용하는 디자인 패턴
- 파이썬 모듈은 기본적으로 모두 싱글턴
- 클래식 싱글턴은 인스턴스가 기존에 생성되어 있는지 확인하고, 그런 경우엔 기존 인스턴스를 반환함
- Borg 싱글턴은 모든 객체에 대해 하나의 공유 상태를 사용함
- 공유 자원과 이미지를 받아올 URL에 접근하기 위해서 Singleton 클래스를 사용했고, 두 스레드는 작업을 올바르게 병렬화했다.